In [1]:
import sys
sys.path.append('../tools/')

import pickle

from feature_format import featureFormat, targetFeatureSplit
from tester import dump_classifier_and_data

/Users/rui/0_my_staff/intro_to_ml/env/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [53]:
financial_features = ['salary', 'deferral_payments', 'total_payments', 'loan_advances', 'bonus', 'restricted_stock_deferred', 'deferred_income', 'total_stock_value', 'expenses', 'exercised_stock_options', 'other', 'long_term_incentive', 'restricted_stock', 'director_fees']
# email_features = ['to_messages', 'email_address', 'from_poi_to_this_person', 'from_messages', 'from_this_person_to_poi', 'shared_receipt_with_poi']
email_features = ['to_messages', 'from_poi_to_this_person', 'from_messages', 'from_this_person_to_poi', 'shared_receipt_with_poi']

# features_list = ['poi', 'salary']
features_list = ['poi'] + financial_features + email_features                      

In [54]:
with open('final_project_dataset.pkl', 'r') as data_file:
    data_dict = pickle.load(data_file)

In [55]:
my_dataset = data_dict

In [56]:
data = featureFormat(my_dataset, features_list, sort_keys=True)
labels, features = targetFeatureSplit(data)

In [57]:
len(features)

145

In [58]:
features[:1]

[array([  2.01955000e+05,   2.86971700e+06,   4.48444200e+06,
          0.00000000e+00,   4.17500000e+06,  -1.26027000e+05,
         -3.08105500e+06,   1.72954100e+06,   1.38680000e+04,
          1.72954100e+06,   1.52000000e+02,   3.04805000e+05,
          1.26027000e+05,   0.00000000e+00,   2.90200000e+03,
          4.70000000e+01,   2.19500000e+03,   6.50000000e+01,
          1.40700000e+03])]

In [59]:
labels[:1]

[0.0]

In [60]:
# from sklearn.model_selection import train_test_split
# features_train, features_test, labels_train, labels_test = train_test_split(
#     features, labels, test_size=0.3, random_state=42
# )

In [68]:
# Naive Bayes
from sklearn.naive_bayes import GaussianNB
estimator = GaussianNB()
param_grid = [{}]

In [61]:
# SVM
from sklearn.svm import SVC
estimator = SVC()
param_grid = [
    {
        'kernel': ['rbf'],
        'C': [1, 10],
    }
]


In [76]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
estimator = RandomForestClassifier(
    n_jobs=-1,
    random_state=42,
)
param_grid = [{
    'n_estimators': [10, 50],
    'criterion': ['gini', 'entropy'],
}]

In [77]:
from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(
    estimator,
    param_grid,
    n_jobs=-1,    
)
clf.fit(features, labels)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=-1, oob_score=False, random_state=42,
            verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid=[{'n_estimators': [10, 50], 'criterion': ['gini', 'entropy']}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [78]:
import pandas as pd
pd.DataFrame(clf.cv_results_)

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_criterion,param_n_estimators,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,0.230725,0.138446,0.875862,0.986218,gini,10,"{u'n_estimators': 10, u'criterion': u'gini'}",1,0.836735,0.989583,0.854167,1.000000,0.937500,0.969072,0.022587,0.004763,0.043941,0.012848
1,0.438118,0.130971,0.868966,1.000000,gini,50,"{u'n_estimators': 50, u'criterion': u'gini'}",2,0.836735,1.000000,0.875000,1.000000,0.895833,1.000000,0.079611,0.014084,0.024537,0.000000
2,0.208113,0.165456,0.868966,0.989655,entropy,10,"{u'n_estimators': 10, u'criterion': u'entropy'}",2,0.877551,0.989583,0.812500,0.989691,0.916667,0.989691,0.029519,0.017330,0.042821,0.000051
3,0.427490,0.124243,0.862069,1.000000,entropy,50,"{u'n_estimators': 50, u'criterion': u'entropy'}",4,0.836735,1.000000,0.875000,1.000000,0.875000,1.000000,0.060968,0.017772,0.018100,0.000000


In [79]:
clf.best_score_

0.87586206896551722

In [80]:
clf.best_params_

{'criterion': 'gini', 'n_estimators': 10}

In [81]:
best_estimator = clf.best_estimator_
best_estimator

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=-1, oob_score=False, random_state=42,
            verbose=0, warm_start=False)

In [82]:
dump_classifier_and_data(best_estimator, my_dataset, features_list)

In [89]:
from sklearn.metrics import r2_score
y = [3, -0.5, 2, 7]
y_pred = [2.5, 0.0, 2, 8]
# y_pred = [-3, 0.5, -2, -7]
print r2_score(y, y_pred)

0.948608137045
